name: link_data \
date: 12/12/2024 \
version: 1.0 \
author: Juliana Agudelo, Johanna Ganglbauer \

**description**: Links LCMS data with general sample description, plots and evaluates correlations.

**changes in comparison to previous version**:

**what needs to be implemented**:


### Packages and parameters


In [ ]:
# install packages
library('ggplot2')
library('corrplot')
library('xlsx')
library('tidyr')
library('dplyr')
library('glue')
library('grid')
library('corrgram')
library('zeallot')
library('ggpubr')
library('gridExtra')
library('gtable')

# restuls folder
results_folder <- 'G:/Shared drives/LOHMANN LAB/STEEP project 4/4_LC MS analysis/STEEP_P3_Slitt_Lab_Juliana_Agudelo/Liver_samples_kansas/plots/'

# thresholds
sigma = 0.5  # threshold for correlation value to be shown in correlation plot (correlations below sigma will not be shown)
cols_per_plot <- 11  # number of proteins shown on each correlation plot
nan_to_zero <- FALSE  # empty cells in proteomics and <MDL in LCMS are treated as zero?
min_coverage <- 0.4  # minimum coverage (not NAN entries in columns) to be included in evaluation
max_char_protein <- 9  # determines maximum number of letters allowed in protein names (used for correlation plots)

Below you find code to automatically read in lcms result files from lcms data analysis script, find samples of interst (relevant samples are indicated in linking_data_file) and put it in a data frame.

In [ ]:
# # read in linking data, LCMS data and put everything together in a data frame
# link_data <- data.frame(read.csv(linking_data_file, na.strings=''))

# # get all results in lcms result folder and put them together in a dataframe
# # curent status: 1 file: todo -> make list!
# list_csv_files = list.files(path = lcms_results_folder, pattern = '*.csv')
# lcms_data <- read.csv(file.path(lcms_results_folder, list_csv_files[1]), na.strings='', stringsAsFactors=FALSE)
# lcms_data <- data.frame(lcms_data)
# for (column in c('Below.Detection.Threshold')){
#     lcms_data[[column]] = as.logical(lcms_data[[column]])
#     lcms_data[is.na(column), column] <- FALSE
# }

# lcms_data$Calculated.Concentration[lcms_data$Below.Detection.Threshold] <- NA

# for (pfas_component in lcms_data$Component.Name){
#     link_data[, pfas_component] <- NA
#     relevant_concentration = lcms_data[lcms_data$Component.Name == pfas_component,]
#     for (row in 1:nrow(link_data)){
#         concentration = relevant_concentration$Calculated.Concentration[relevant_concentration$Sample.Name == link_data[row, 'sample_name_in_lcms']]
#         link_data[row, pfas_component] = concentration
#     }
# }

Very specific case: special input file for LCMS (not generalized), relevant for one concrete example. \
Read in data and replace '<MDL' with NA \
Append lcms data to original 'linking_data' data frame

In [ ]:
# lcms_data <- read.xlsx(
#     lcms_results_file, lcms_results_sheet_name, rowIndex = 18:21, colIndex = 1:14, stringsAsFactors = FALSE
#     )
# lcms_data <- lcms_data %>% mutate_all(~ifelse(. == "<MDL", NA, .))

# for (pfas_component in colnames(lcms_data)){
#     if (pfas_component == 'ng.g') {
#         next
#         }
#     for (row in 1:nrow(link_data)){
#         concentration = as.numeric(as.character(lcms_data[[pfas_component]][lcms_data$ng.g == link_data[row, 'sample_name_in_lcms']]))
#         link_data[row, pfas_component] = concentration
#     }
# }

### Option 2
Relevant data + PFAS data in one file, proteomics somewhere else
(1) read in and clean 'general input table'
(2) read in and clean proteomics data

In [ ]:
# read in relevant rows
linking_data <- read.xlsx(
    'G:/Shared drives/LOHMANN LAB/STEEP project 4/4_LC MS analysis/STEEP_P3_Slitt_Lab_Juliana_Agudelo/Liver_samples_kansas/Liver_samples_kansas.xlsx',
    'Use this', rowIndex = 1:96, colIndex = append(c(5, 7, 14, 26, 29, 37, 38, 49, 50, 51, 55), seq(58, 82, 1)), stringsAsFactors = FALSE
)

# data cleaning
# replace 1 month age with 0 (years)
linking_data$Age <- replace(linking_data$Age, linking_data$Age=='1 month', 0)

# replace 'Unk' with NA
linking_data$Body.Mass.Index..BMI. <- replace(linking_data$Body.Mass.Index..BMI., linking_data$Body.Mass.Index..BMI.=='Unk', NA)

# replace 'Unknown' with NA
linking_data$Specimen.Collection.Year <- replace(linking_data$Specimen.Collection.Year, linking_data$Specimen.Collection.Year=='Unknown', NA)

# selct only rows which contain either PFAS Analysis or Proteomics Analysis
linking_data <- linking_data[linking_data$Processed.for.PFAS.Targeted.LC.MS == 'YES' | linking_data$Processed.for.Proteomics.Analysis == 'YES', ]

# Read in proteomics data and append to linking_data

In [ ]:
# read in proteomics data
proteomics_data <- read.csv(
  'G:/Shared drives/LOHMANN LAB/STEEP project 4/4_LC MS analysis/STEEP_P3_Slitt_Lab_Juliana_Agudelo/Liver_samples_kansas/202411_liver_samples_kansas_proteins.csv',
  stringsAsFactors = FALSE
  )

available_sample_names <- colnames(proteomics_data)  # get column headers from proteomics data
number_of_linking_data_rows = nrow(linking_data)  # number of rows (samples) in linking data

# make protein descriptions shorter -> otherwise caption in correlation plot takes up too much space
for (row_index in seq_along(proteomics_data$PG.ProteinNames)){
    protein_name <- proteomics_data[row_index, "PG.ProteinNames"]
    if (nchar(protein_name) > max_char_protein){
        proteomics_data[row_index, "PG.ProteinNames"] <- substr(protein_name, start=1, stop=max_char_protein)
    }
}

# loop over proteins and append value to correct row of linking data samples
for(protein in proteomics_data$PG.ProteinNames){
    # get protein concentrations for all samples and protein name
    protein_concentrations <- proteomics_data[proteomics_data$PG.ProteinNames == protein, ]
    protein_name <- make.names(protein)
    # loop over samples and append protein concentration column to linking data
    for(row in 1:number_of_linking_data_rows){
        sample_name_in_linking_data <- make.names(linking_data[row, 'Site...Specimen.Number'])
        mask_sample_name_in_proteomics_data <- grepl(sample_name_in_linking_data, available_sample_names)
        if(any(mask_sample_name_in_proteomics_data)){
            sample_name_in_proteomics_data <- available_sample_names[mask_sample_name_in_proteomics_data]
            linking_data[row, protein_name] <- protein_concentrations[[sample_name_in_proteomics_data]]
        } 
    }
}

# Data Cleaning
- rename columns of dataframe
- delete columns with too many NA values (< min coverage>)
- make sure all numbers are interpreted as numbers
- shorten protein descriptions to 10 letters


In [ ]:
# compute data coverage of each row (how many data entries compared to NaNs)
coverage <- linking_data %>% summarise_all(~ number_of_linking_data_rows - sum(is.na(.))) / number_of_linking_data_rows

# delete columns with low coverage
linking_data <- linking_data[,coverage >= min_coverage]

# rename columns
colnames(linking_data)[colnames(linking_data) == 'BCA.Total.Protein..mg.mL.or.g.L.'] <- 'Proteins'
colnames(linking_data)[colnames(linking_data) == 'Total.Lipids..g.g.'] <- 'Lipids'
colnames(linking_data)[colnames(linking_data) == 'Liver.Fat.Content....'] <- 'Liverfat'
colnames(linking_data)[colnames(linking_data) == 'Body.Mass.Index..BMI.'] <- 'BMI'
colnames(linking_data)[colnames(linking_data) == 'X.U.2211.PFAS..ng.g.'] <- 'PFAS'

# Get all PFAS column names
all_columns <- colnames(linking_data)
pfas_subset <- all_columns[grepl('(PF|FOSA|FTS)', all_columns)]
pfas_subset <- pfas_subset[!grepl('(Processed.for.PFAS.Targeted.LC.MS.|rotein|RNA|_HUM)', pfas_subset)]

# get all protein column names
protein_subset <- all_columns[seq(match(pfas_subset[length(pfas_subset)],all_columns), length(all_columns), 1)]
protein_subset <- protein_subset[seq(2, length(protein_subset), 1)]

# Set all pfas values below MDL to zero (it is important that rows where no PFAS analysis was conducted remain NaN)
mdls <- read.xlsx(
  'G:/Shared drives/LOHMANN LAB/STEEP project 4/4_LC MS analysis/STEEP_P3_Slitt_Lab_Juliana_Agudelo/Liver_samples_kansas/20241212_liver_samples_kansas_lcms.xlsx',
  'Detection Threshold', colIndex = c(1,6), stringsAsFactors = FALSE
)

colnames(mdls) <- c('pfas', 'mdl')

for (pfas_component in pfas_subset){
  if(pfas_component != 'PFAS'){
    pfas_identifier <- gsub("L.", "L-", pfas_component)
    pfas_identifier <- gsub("Br.", "Br-", pfas_identifier)
    pfas_identifier <- gsub("X6.2.", "6:2 ", pfas_identifier)
    mdl <- mdls[mdls['pfas']==pfas_identifier, 'mdl']
  }
  else{
    mdl<-0
    }
  linking_data[linking_data$Processed.for.PFAS.Targeted.LC.MS=='YES', pfas_component][
    is.na(linking_data[linking_data$Processed.for.PFAS.Targeted.LC.MS=='YES', pfas_component])
  ] <- mdl / sqrt(2)
}

# delete rows which indicate if total protein was analysed/if pfas was analyzed
linking_data <- linking_data[,-c(3, 5)]

# summarize all column headers, which will be relevant for further analysis
relevant_columns <- append(c('Proteins', 'Lipids', 'Liverfat', 'BMI', 'Age'), pfas_subset)
relevant_columns_and_proteins <- append(relevant_columns, protein_subset)

# convert to numeric type
for (column in relevant_columns_and_proteins){
  linking_data[, column] <- as.numeric(as.character(linking_data[, column]))
}

IRdisplay::display(linking_data)


# boxplots dependent on health status and gender

### Option one: two groups (male/female) and (normal health, NAFLD) and t.test
### Option two: four groups (male+normal health, male + NAFLD, female + normal health, female + NAFLD) with one-way ANOVA

In [ ]:
# for (column in relevant_columns){
#     p_value <- t.test(linking_data[linking_data$Sex=='Male', column], linking_data[linking_data$Sex=='Female', column])$p.value
#     p_value_string <- glue('p-value:{round(p_value,3)}')
#     png(glue('{results_folder}boxplot_gender_{column}.png'))
#     gg <- ggplot(data=linking_data, aes_string(x='Sex', y=column, group='Sex')) + geom_boxplot() + 
#     annotation_custom(grobTree(textGrob(p_value_string, x=0.4,  y=0.95, hjust=0)))
#     print(gg)
#     dev.off()
# }

# for (column in relevant_columns){
#     p_value <- t.test(linking_data[linking_data$Health.Status=='Normal', column], linking_data[linking_data$Health.Status=='NAFLD', column])$p.value
#     p_value_string <- glue('p-value:{round(p_value,3)}')
#     png(glue('{results_folder}boxplot_health_status_{column}.png'))
#     gg <- ggplot(data=linking_data, aes_string(x='Health.Status', y=column, group='Health.Status')) + geom_boxplot() + 
#     annotation_custom(grobTree(textGrob(p_value_string, x=0.4,  y=0.95, hjust=0)))
#     print(gg)
#     dev.off()
# }

    # png(glue('{results_folder}scatterplot_age_{column}.png'))
    # gg <- ggplot(data=linking_data, aes_string(x='Age', y=column, shape='Sex')) + geom_point(size=6)
    # print(gg)
    # dev.off()



In [ ]:
# useful: https://www.sthda.com/english/wiki/one-way-anova-test-in-r

# make new column combining information from Sex and Health.Status
four_group_linking_data <- linking_data
for (row in rownames(linking_data)){
    four_group_linking_data[row, 'Sex.And.Health'] <- paste0(
        linking_data[row, 'Sex'], "_", linking_data[row, 'Health.Status']
    )
}

# only use relevant columns not proteins
four_group_linking_data <- four_group_linking_data[append(relevant_columns, 'Sex.And.Health')]
plot_list <- list()
i <- 0

# instead of boxplots, you can also make points and averages (see link above)
for (column in relevant_columns){
    # # Compute the analysis of variance
    aov_calc <- aov(four_group_linking_data[,column] ~ Sex.And.Health, data = four_group_linking_data)
    # Summary of the analysis
    print(column)
    print(summary(aov_calc))
    if(summary(aov_calc)[[1]][["Pr(>F)"]][[1]] <= 0.05){
        print(TukeyHSD(aov_calc))
    }
    # p_value <- t.test(linking_data[linking_data$Sex=='Male', column], linking_data[linking_data$Sex=='Female', column])$p.value
    # p_value_string <- glue('p-value:{round(p_value,3)}')
    g <- ggplot(
        data=four_group_linking_data, aes_string(x='Sex.And.Health', y=column, group='Sex.And.Health', fill='Sex.And.Health')
        ) + geom_boxplot() + theme(legend.position='bottom')
    if(i==1){
        legend <- gtable_filter(ggplotGrob(g), "guide-box")
    }
    plot_list[[i + 1]] <- ggplotGrob(g + theme(axis.title.x = element_blank(), axis.text.x = element_blank(), legend.position='none'))
    # annotation_custom(grobTree(textGrob(p_value_string, x=0.4,  y=0.95, hjust=0)))
    i <- i + 1
}

plot_list <- append(plot_list, list(legend))
lay <- rbind(c(1,2,3),
             c(4,5,6),
             c(7,8,9),
             c(10,11,12),
             c(13,13,13))

png(glue('{results_folder}boxplots_gender_and_health_status.png'))
grid.arrange(grobs=plot_list, layout_matrix=lay)
dev.off()

Function to compute correlation and filter certain values. \
Plot correlation data only from fields which are not NaN, not 1 and the magnitude is greater than sigma.
Plot correlations from each relevant point (not 1, not NaN and magnitude greater than sigma)

In [ ]:
corr_simple <- function(xdata, ydata, sig){
  # run a correlation and test its significance - possible for different dimensions in x and y
  # correlation excluding NA
  mtx_corr <- cor(xdata, ydata, method = "spearman", use='pairwise.complete.obs')
  # p value, element-wise with column of x data and column of y data
  mtx_p <- mtx_corr # initialize
  # loop over x and y in correlation matrix
  for(x in colnames(xdata)){
    for(y in colnames(ydata)){
      mtx_p[x, y] <- NA
      mtx_p[x,y] <- cor.test(xdata[[x]], ydata[[y]], alternative = "two.sided")$p.value
    }
  }
  # delete all columns where no correlation is greater than sigma (looking at absolute values)
  mtx_p <- mtx_p[, apply(mtx_corr, 2, function(x) any(abs(x) > sigma))]
  mtx_corr <- mtx_corr[, apply(mtx_corr, 2, function(x) any(abs(x) > sigma))]

  # optional delete all columns where no p-value is greather than 0.05
  mtx_corr <- mtx_corr[, apply(mtx_p, 2, function(x) any(x <= 0.05))]
  mtx_p <- mtx_p[, apply(mtx_p, 2, function(x) any(x <= 0.05))]

  return(list(mtx_corr, mtx_p))
  }

plot_corr <- function(mtx_corr, mtx_p, results_folder, n){
  # correlation plot possible for different dimensions in x and y
  # prepare text for each field of the corrplot (correlation value, significance)
  mtx_text <- mtx_corr
  for (row in rownames(mtx_corr)){
    for (column in colnames(mtx_corr)){
      # correlation value
      r <- formatC(mtx_corr[row, column], digits = 2, format = "f")
      # stars based on p value
      est <- mtx_p[row, column]
      stars <- ifelse(est < 5e-4, "***",
                ifelse(est < 5e-3, "**", 
                  ifelse(est < 5e-2, "*", "")))

      # save to text dataframe
      mtx_text[row, column] <- NA
      mtx_text[row, column] <- paste0(r, "\n", stars)
    }
  }

  # plot correlations visually
  png(glue('{results_folder}correlation_plot_{n}.png'))
  corrplot(
    mtx_corr, method='color', tl.col="black", addgrid.col='lightgrey',
    col=colorRampPalette(colors=c("red", "salmon", "white", "royalblue", "navy"))(100),
    )$corrPos -> hc
  text(hc$x, hc$y, mtx_text)
  dev.off()
}

customized_panel_function <- function(x, y, corr = NULL, col.regions, ...){
  # source for plot from pfas and whaltes paper: 
  # https://stackoverflow.com/questions/19012529/correlation-corrplot-configuration
  corr <- cor(x, y, method="spearman", use="pairwise.complete.obs")
  est <- cor.test(x, y, alternative = "two.sided")$p.value
  stars <- ifelse(est < 5e-4, "***", 
                  ifelse(est < 5e-3, "**", 
                         ifelse(est < 5e-2, "*", "")))
  ncol <- 14
  pal <- col.regions(ncol)
  col.ind <- as.numeric(cut(corr, breaks = seq(from = -1, to = 1, length = ncol + 1), include.lowest = TRUE))
  usr <- par("usr")
  rect(usr[1], usr[3], usr[2], usr[4], col = pal[col.ind], border = NA)
  box(col = "lightgray")
  on.exit(par(usr))
  par(usr = c(0, 1, 0, 1))
  r <- formatC(corr, digits = 2, format = "f")
  cex.cor <- .8/strwidth("-X.xx")
  fonts <- ifelse(stars != "", 2,1)
  # option 1: stars:
  text(0.5, 0.4, paste0(r, "\n", stars), cex = cex.cor)
}

plot_single_corrs <-function(mtx_corr, mtx_p, link_data, results_folder, sigma){
  for (pfas in rownames(mtx_corr)){
    xdata <- link_data[[pfas]]
    for (protein in colnames(mtx_corr)){
      correlation <- mtx_corr[pfas, protein]
      p <- mtx_p[pfas, protein]
      if (is.na(correlation)){
        next
      } else{
        if (abs(correlation) > sigma && correlation != 1 && p <= 0.05){
          ydata <- link_data[[protein]]
          x_name <- gsub('\\.', '_', pfas)
          y_name <- gsub('\\.', '_', make.names(protein))
          png(glue('{results_folder}correlation_{x_name}_{y_name}.png'))
          gg <- ggplot(data=link_data, aes(x=xdata, y=ydata)) + geom_point() + geom_smooth(method='lm', se=TRUE, linetype='dashed') + 
          labs(x=glue('{x_name} [ng/g]'), y=glue('{y_name} [?]')) + theme_minimal()
          print(gg)
          dev.off()
        }
      }
    }
  }
}

Make (PFAS vs. proteomics) correlation plots

In [ ]:
# plot correlation matrix of pfas and general information both ways
# evaluate correlation between PFAS and general data
c(mtx_corr_basic, mtx_p_basic) %<-% corr_simple(xdata=linking_data[relevant_columns], ydata=linking_data[relevant_columns], sig=sigma)

# plot correlation between PFAS and general data
png(glue('{results_folder}correlation_plot.png'))
corrgram(linking_data[relevant_columns], type="data", lower.panel=customized_panel_function, upper.panel=NULL)
dev.off()

# plot_corr(mtx_corr=mtx_corr_basic, mtx_p=mtx_p_basic, results_folder=results_folder, n=999)

# evaluate correlation matrix of pfas and general information vs. proteins
c(mtx_corr_proteins, mtx_p_proteins) %<-% corr_simple(
  xdata=linking_data[relevant_columns], ydata=linking_data[protein_subset], sig=sigma
  )

# plot correlation matrix of pfas and general information vs. proteins
for (n in seq(from=1, to=floor(ncol(mtx_corr_proteins)/cols_per_plot), by=1)){
  plot_corr(
    mtx_corr=mtx_corr_proteins[, c(seq((n-1) * cols_per_plot, n * cols_per_plot, 1))],
    mtx_p=mtx_p_proteins[, c(seq((n-1) * cols_per_plot, n * cols_per_plot, 1))],
    results_folder=results_folder, n=n
    )
}

plot_corr(
  mtx_corr=mtx_corr_proteins[, c(seq((n) * cols_per_plot, ncol(mtx_corr_proteins), 1))],
  mtx_p=mtx_p_proteins[, c(seq((n) * cols_per_plot, ncol(mtx_corr_proteins), 1))],
  results_folder=results_folder, n=n + 1
  )

# plot correlations with correlation value above sigma and p <= 5 %
plot_single_corrs(mtx_corr=mtx_corr_basic, mtx_p=mtx_p_basic, link_data=linking_data[relevant_columns_and_proteins], results_folder=results_folder, sigma=sigma)
plot_single_corrs(mtx_corr=mtx_corr_proteins, mtx_p=mtx_p_proteins, link_data=linking_data[relevant_columns_and_proteins], results_folder=results_folder, sigma=sigma)